# Домашнее задание 1. Извлечение ключевых слов

Мини-корпус с уже выделенной разметкой ключевых слов + самостоятельной разметкой ключевых слов. Источник текстов -- новостной сайт [вести.ру](https://www.vesti.ru/); ключевые слова представлены в виде тегов, по которым можно сделать поиск похожих статей.



In [1]:
import pandas as pd
df = pd.read_csv('kw_corpus.csv')
df.head()

,text,url,keywords by source,keywords by me,sum of keywords
0,"Администрация мессенджера WhatsApp, принадлежа...",https://www.vesti.ru/hitech/article/2482035,"технологии, новости, мессенджеры, WhatsApp, пл...","WhatsApp, Индия, денежные переводы",NaN
1,Вслед за Евросоюзом дополнительные санкции в о...,https://www.vesti.ru/article/2482032,"общество, в мире, политика, новости, Белорусси...","Канада, Беларусь, санкции, права целовека",NaN
2,Естественная убыль населения в период с января...,https://www.vesti.ru/article/2482034,"общество, новости, смертность, статистика, Рос...","естественная убыль населения, Росстат, смертно...",NaN
3,Россияне стали более требовательны к местным о...,https://www.vesti.ru/article/2481993,"общество, политика, новости, Владимир Путин, в...","выборы, волонтерство, доверие, технологичность",NaN
4,"""Медленный Джо"", так Байдена всю дорогу называ...",https://www.vesti.ru/article/2481999,"политика, новости, президент, Джозеф Байден, С...","Байден, Трамп, подсчет голосов, голосование",NaN


Добавим последний столбец (эталон разметки) -- объединение ручной разметки и исходной

In [2]:
kw_source = df['keywords by source'].tolist()
kw_author = df['keywords by me'].tolist()
kw_sum = [', '.join(list(set(el.split(', ') + kw_author[i].split(', ')))) for i, el in enumerate(kw_source)]
df['sum of keywords'] = kw_sum
df.head()

,text,url,keywords by source,keywords by me,sum of keywords
0,"Администрация мессенджера WhatsApp, принадлежа...",https://www.vesti.ru/hitech/article/2482035,"технологии, новости, мессенджеры, WhatsApp, пл...","WhatsApp, Индия, денежные переводы","мессенджеры, платежи, WhatsApp, hi-tech, новос..."
1,Вслед за Евросоюзом дополнительные санкции в о...,https://www.vesti.ru/article/2482032,"общество, в мире, политика, новости, Белорусси...","Канада, Беларусь, санкции, права целовека","права целовека, Белоруссия, Александр Лукашенк..."
2,Естественная убыль населения в период с января...,https://www.vesti.ru/article/2482034,"общество, новости, смертность, статистика, Рос...","естественная убыль населения, Росстат, смертно...","Росстат, естественная убыль населения, коронав..."
3,Россияне стали более требовательны к местным о...,https://www.vesti.ru/article/2481993,"общество, политика, новости, Владимир Путин, в...","выборы, волонтерство, доверие, технологичность","технологичность, Владимир Путин, новости, воло..."
4,"""Медленный Джо"", так Байдена всю дорогу называ...",https://www.vesti.ru/article/2481999,"политика, новости, президент, Джозеф Байден, С...","Байден, Трамп, подсчет голосов, голосование","голосование, Трамп, Америка, выборы в США 2020..."


## Нормализация текстов

In [139]:
#pip install git+https://github.com/aatimofeev/spacy_russian_tokenizer.git

In [140]:
#pip install pymorphy2

In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from spacy.lang.ru import Russian
from spacy_russian_tokenizer import RussianTokenizer, MERGE_PATTERNS


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
morph = MorphAnalyzer()
def normalize_text(text):
    # убираем стоп-слова
    text = ' '.join([word for word in text.lower().split() if word not in stopwords.words('russian')])
    # разбиваем на токены
    nlp = Russian()
    doc = nlp(text)
    russian_tokenizer = RussianTokenizer(nlp, MERGE_PATTERNS)
    nlp.add_pipe(russian_tokenizer, name='russian_tokenizer')
    doc = nlp(text)
    tokens = [token.text for token in doc if token.text not in ',.!?-();:""«»—_–#' and '.' not in token.text]
    return ' '.join([morph.parse(word)[0].normal_form for word in tokens])

In [5]:
df['normalized_text'] = df['text'].apply(normalize_text)

## RAKE

In [6]:
#pip install python-rake

In [7]:
import RAKE

In [8]:
stop = stopwords.words('russian')

In [9]:
rake = RAKE.Rake(stop)

In [44]:
rake_kw = []
for i, row in df.iterrows():
    rake_words = [w[0] for w in rake.run(df['text'][i], maxWords=2, minFrequency=1)]
    list_kw = []
    for word in rake_words:
        w = ' '.join([morph.parse(word)[0].normal_form for word in word.split()])
        list_kw.append(w)
    list_kw = ', '.join(list_kw)
    rake_kw.append(list_kw)

In [49]:
df['RAKE'] = rake_kw

## Summa


In [12]:
#pip install summa

In [13]:
from summa import keywords

In [14]:
summa_kw = []
for i, row in df.iterrows():
    summa_kw.append(', '.join([w[0] for w in keywords.keywords(df['normalized_text'][i], language='russian', additional_stopwords=stop, scores=True)]))

In [15]:
df['summa'] = summa_kw

## TextRank

In [16]:
from gensim.summarization import keywords as kw

In [17]:
textrank_kw = []
for i, row in df.iterrows():
    textrank_kw.append(', '.join([w[0] for w in kw(df['normalized_text'][i], pos_filter=[], scores=True)]))

In [18]:
df['TextRank'] = textrank_kw

## Морфосинтаксические шаблоны

Будем использовать шаблоны ADJ + NOUN, NOUN + NOUN, NOUN (также в качестве pos-теггера для улучшения качества работы TextRank)

In [19]:
def pos(kw):
    res = []
    list_kw = kw.split(', ')
    for word in list_kw:
        if len(word.split()) == 1:
            res.append(morph.parse(word)[0].tag.POS)
        else:
            res.append([morph.parse(w)[0].tag.POS for w in word.split()])
    
    return res

In [20]:
templates = ['NOUN', "['ADJF', 'NOUN']", "['NOUN', 'NOUN']"]

## RAKE + pos-tagging

In [46]:
rake_kw_pos = []
for k, kw in enumerate(rake_kw):
    pos_tagging = pos(kw)
    kw = kw.split(', ')
    text_kw = [kw[i] for i, el in enumerate(pos_tagging) if str(el) in templates]
    rake_kw_pos.append(', '.join(text_kw))

df['RAKE + pos-tagging'] = rake_kw_pos

## summa + pos-tagging

In [22]:
summa_kw_pos = []
for k, kw in enumerate(summa_kw):
    pos_tagging = pos(kw)
    kw = kw.split(', ')
    text_kw = [kw[i] for i, el in enumerate(pos_tagging) if str(el) in templates]
    summa_kw_pos.append(', '.join(text_kw))

In [23]:
df['summa + pos-tagging'] = summa_kw_pos

## TextRank + pos-tagging

In [24]:
textrank_kw_pos = []
for k, kw in enumerate(textrank_kw):
    pos_tagging = pos(kw)
    kw = kw.split(', ')
    text_kw = [kw[i] for i, el in enumerate(pos_tagging) if str(el) in templates]
    textrank_kw_pos.append(', '.join(text_kw))

df['TextRank + pos-tagging'] = textrank_kw_pos

In [25]:
df.head()

,text,url,keywords by source,keywords by me,sum of keywords,normalized_text,RAKE,summa,TextRank,RAKE + pos-tagging,summa + pos-tagging,TextRank + pos-tagging
0,"Администрация мессенджера WhatsApp, принадлежа...",https://www.vesti.ru/hitech/article/2482035,"технологии, новости, мессенджеры, WhatsApp, пл...","WhatsApp, Индия, денежные переводы","мессенджеры, платежи, WhatsApp, hi-tech, новос...",администрация мессенджер whatsapp принадлежащи...,"которая тестироваться, запуск сервис, позволяю...","индия, мессенджер whatsapp, платёжный, upi, ba...","индия, мессенджер whatsapp, функция, платежныи...","запуск сервис, дебетовую карта, блоге компания...","индия, банк, перевод, деньга, время, пользоват...","индия, функция, платежныи, банк, деньга, местн..."
1,Вслед за Евросоюзом дополнительные санкции в о...,https://www.vesti.ru/article/2482032,"общество, в мире, политика, новости, Белорусси...","Канада, Беларусь, санкции, права целовека","права целовека, Белоруссия, Александр Лукашенк...",вслед евросоюз дополнительный санкция отношени...,"санкции связать, политической ситуация, канаде...","санкция отношение, ес, канада, мид, недовольст...","санкция отношение, канада, мид, недовольство, ...","политической ситуация, санкциях ес, своей стра...","санкция отношение, ес, канада, мид, недовольст...","санкция отношение, канада, мид, недовольство, ..."
2,Естественная убыль населения в период с января...,https://www.vesti.ru/article/2482034,"общество, новости, смертность, статистика, Рос...","естественная убыль населения, Росстат, смертно...","Росстат, естественная убыль населения, коронав...",естественный убыль население период январь сен...,"прошлом года, сайте ведомство, росстате отмеча...","сентябрь, тысяча человек, росстат, год состави...","тысяча человек, сентябрь, росстат, год состави...","прошлом года, сайте ведомство, период, январь,...","сентябрь, тысяча человек, росстат, смертность,...","тысяча человек, сентябрь, росстат, ведомство, ..."
3,Россияне стали более требовательны к местным о...,https://www.vesti.ru/article/2481993,"общество, политика, новости, Владимир Путин, в...","выборы, волонтерство, доверие, технологичность","технологичность, Владимир Путин, новости, воло...",россиянин стать требовательный местный орган в...,"россияне стать, одержал побед, который прошеть...","выборы следующий год, человек, памфилов, урове...","выборы, тот, человек, год, которыи, сам, урове...","новых лица, ходе разговор, главой государство,...","человек, памфилов, результат, избирательный си...","выборы, человек, год, которыи, результат, наро..."
4,"""Медленный Джо"", так Байдена всю дорогу называ...",https://www.vesti.ru/article/2481999,"политика, новости, президент, Джозеф Байден, С...","Байден, Трамп, подсчет голосов, голосование","голосование, Трамп, Америка, выборы в США 2020...",медленный джо байден дорога называть трамп сег...,"знаменитый лас-вегас, медленный джо, стать пер...","трамп сегодня, это, байден, президент, демокра...","это, трамп сегодня, баиден, которыи, президент...","знаменитый лас-вегас, медленный джо, пенсильва...","байден, президент, демократия, подсчёт голос, ...","баиден, которыи, президент, подсчет голос, чел..."


## Точность, полнота и f-мера относительно эталона

In [47]:
def tp(kw_gold, kw_counted):
    a = kw_gold.lower().split(', ')
    b = kw_counted.split(', ')
    c = []
    for i in a:
        for j in b:
            if i == j:
                c.append(i)
                break
    return len(c)

def fp(kw_gold, kw_counted):
    # резмечены методом как ключевые слова, но отсутствуют в эталонном списке
    right_kw = tp(kw_gold, kw_counted)
    return len(kw_counted.split(', ')) - right_kw

def fn(kw_gold, kw_counted):
    # количество слов из эталонного списка, которые ошибочно не были размечены как ключевые
    right_kw = tp(kw_gold, kw_counted)
    return len(kw_gold.split(', ')) - right_kw

In [53]:
def precision(kw_gold, kw_counted):
    return round(tp(kw_gold, kw_counted)/(tp(kw_gold, kw_counted)+fp(kw_gold, kw_counted)), 4)

def recall(kw_gold, kw_counted):
    return round(tp(kw_gold, kw_counted)/(tp(kw_gold, kw_counted)+fn(kw_gold, kw_counted)), 4)

def f_measure(kw_gold, kw_counted):
    if precision(kw_gold, kw_counted)+recall(kw_gold, kw_counted) == 0:
        return 0
    else:
        return round(2*precision(kw_gold, kw_counted)*recall(kw_gold, kw_counted)/(precision(kw_gold, kw_counted)+recall(kw_gold, kw_counted)), 4)

In [28]:
from prettytable import PrettyTable

In [54]:
for i, row in df.iterrows():
    print('Качество выделения ключевых слов для текста ', (i+1))
    x = PrettyTable()
    x.field_names = [' ', 'RAKE', 'summa', 'TextRank', 
                                      'RAKE + pos-tagging', 'summa + pos-tagging', 
                                      'TextRank + pos-tagging']
    gold = row['sum of keywords']
    x.add_row(['precision', precision(gold, row['RAKE']), 
               precision(gold, row['summa']), 
               precision(gold, row['TextRank']), 
               precision(gold, row['RAKE + pos-tagging']), 
               precision(gold, row['summa + pos-tagging']), 
               precision(gold, row['TextRank + pos-tagging'])])
    x.add_row(['recall', recall(gold, row['RAKE']), 
               recall(gold, row['summa']), 
               recall(gold, row['TextRank']), 
               recall(gold, row['RAKE + pos-tagging']), 
               recall(gold, row['summa + pos-tagging']), 
               recall(gold, row['TextRank + pos-tagging'])])
    x.add_row(['f-measure',  f_measure(gold, row['RAKE']), 
               f_measure(gold, row['summa']), 
               f_measure(gold, row['TextRank']), 
               f_measure(gold, row['RAKE + pos-tagging']),
               f_measure(gold, row['summa + pos-tagging']), 
               f_measure(gold, row['TextRank + pos-tagging'])])
    print(x, '\n')

Качество выделения ключевых слов для текста  1
+-----------+--------+--------+----------+--------------------+---------------------+------------------------+
|           |  RAKE  | summa  | TextRank | RAKE + pos-tagging | summa + pos-tagging | TextRank + pos-tagging |
+-----------+--------+--------+----------+--------------------+---------------------+------------------------+
| precision | 0.0294 | 0.0556 |  0.0588  |        0.0         |        0.1429       |         0.0909         |
|   recall  | 0.125  | 0.125  |  0.125   |        0.0         |        0.125        |         0.125          |
| f-measure | 0.0476 | 0.077  |   0.08   |         0          |        0.1334       |         0.1053         |
+-----------+--------+--------+----------+--------------------+---------------------+------------------------+ 

Качество выделения ключевых слов для текста  2
+-----------+--------+-------+----------+--------------------+---------------------+------------------------+
|           |  RA

## Описание ошибок и методы решения

Выбранные метрики показывают низкое качество (с учетом частеречных шаблонок несколько выше) работы методов по автоматическому извлечению ключевых слов. Это может объясняться следующими причинами:



1.   Автоматический подсчет (по полному совпадению) полноты, точности и ф-меры. Это значит, например, что для первого текста в эталонную разметку входили слова 'мессенджеры' и 'whatsapp', а в разметку TextRank словосочетание 'мессенджер whatsapp', которое не засчитывается, как правильное (при ручном определении true positive, false positive и false negative, скорее всего, качество было бы выше)
2.   Слишком мало слов в эталонной разметке или недостаточная длина некоторых текстов в корпусе
3.   Ошибки и неточности в pos-тэггере: разметка 'который' в качестве существительного, практически никогда не используемая в речи лемма 'деньга' и т.д. 

Еще описание ошибок автоматического выделения слов:

- Для метода RAKE были использованы нелемматизированные и нетокенизированные тексты, так как в противном случае для некоторых текстов метод совсем не выделял ключевые слова. Далее уже полученные ключевые слова были лемматизированы, чтобы улучшить качество при сравнении слов с эталоном, но несмотря на это многие ключевые слова выглядят странно (не согласуются между собой и т.д.)
- Метод RAKE выделяет большое количество именных сущностей, которые, хотя и могут быть важными для некоторых текстов, в данном корусе привели к неправильному выбору ключевых слов: в тексте 6 идет перечисление людей, которые высказали свои соболезнования, и метод выделяет их как главные существительные текста. Можно попробовать избавиться от этой ошибки путем удаления именованных сущностей из текста, или удалением тех именнованных сущностей, которые встречаются в тексте только один раз
- Ожидаемо, автоматические методы не выделяют ключевые слова, которые связаны с общим содержанием текста или передают его тематику: такую задачу мог бы выполнить прагматический анализ текста. Можно использовать методы тематического моделирования, классифицировать по теме наиболее частотные слова в тексте или сами тексты, тогда в списке ключевых слов будут слова, которые не встречались в тексте, но отражают его содержание. 
- Автоматические методы часто выделяют глагольные группы в качестве ключевых слов: говорит Путин, выросло доверие и т.д. Однако в русском языке при обозначении ключевых слов носителями чаще используются имена, а для обозначения действий отглагльные существительные --> можно заменять выделенные словосочетания с глаголом словосочетниями типа сущ + сущ.
- Для больших текстов автоматические методы выбирают много ключевых слов, но большинство из них оказываются нерелевантными или синонимичными (*крик, гнев, нож* для политической новости про выборы в США) -- как средство для решения: отбирать топ ключевых слов, а не включать в список все слова, выделенные моделью